# Demo Jupyter Notebook für das Blockseminar 2023

**Autoren**

- Ulf A. Hamster (orcid: [0000-0002-0440-4868](https://orcid.org/0000-0002-0440-4868))


**Zusammenfassung**

Wir zeigen, wie Sequential Quadratic Programming (SQP) verwendet werden kann, um Bewertungen (goodness score) mit Ähnlichkeitsmetriken (similarity scores) abzuwägen, um ein diversifizierte Auswahl guter Satzbelege zu finden.

# Das Duale Optimierungsproblem
Die Idee, einen Vektor vorteilhafter Merkmalsausprägungen mit 
mit einer Matrix ungünstiger paarweiser Abhängigkeiten abzuwägen wurde aus dem Portfolio Selection Problem entlehnt [[Mar52](https://doi.org/10.2307/2975974)] bzw. allgemeiner dem Quadratischen Optimierungsproblem [[Mar56](https://doi.org/10.1002/nav.3800030110); [FW56](https://doi.org/10.1002/nav.3800030109)].
In unserem Fall werden die Bewertung einzelner Satzbeispiele (''goodness core'') und die Jaccard-Ähnlichkeitssmetriken zwischen diesen Satzbeispielen bewertet (''similarity score'').

## Problemformulierung
Sei 

- $N \in \mathbb{N}_{>0}$ die Anzahl der Satzbelege, 
- $u \in \{x \in \mathbb{R} | 1 \geq x > 0\}$ die obere Schranke,
- $w_i \in \{x \in \mathbb{R} | 1 \geq x \geq 0\}$ die neue Gewichtung und
- $s_i \in \{x \in \mathbb{R} | 1 \geq x \geq 0\}$ der ''goodness score'' des $i$-ten Satzbelegs,
- $Q_{i,j} \in \{x \in \mathbb{R} | 1 \geq x \geq 0\}$ der (Jaccard) ''similarity score'' zwischen dem $i$-ten und $j$-ten Satzbelegs,
- $\lambda \in \{x \in \mathbb{R} | x > 0\}$ die Präferenz eher die Diversität zu maximieren (Ähnlichkeit zu minimieren) statt die ''goodness'' zu maximieren,

dann wird das quadratische Optierungsproblem wie folgt formuliert:

$$
\min_{w_1, ..., w_N}  \quad 
        -\underbrace{\sum_{i=1}^N w_i s_i}_\text{goodness}
        + \lambda \underbrace{\sqrt{\sum_{i=1}^N \sum_{j=1}^N w_i Q_{i,j} w_j}}_\text{similarity}
$$

<div></div>

$$
{\rm s.t. } \quad
 \sum_{i=1}^N w_i = 1
$$

<div></div>

$$
 w_i \geq 0 \quad \forall i
$$

<div></div>

$$
 w_i \leq u \quad \forall i
$$


Wir verwenden $w_i^{(0)}=\frac{1}{N} \; \forall i$ als Startwerte der Optimeirung, und setzen die obere Schranke der Gewichtungen auf 
$u = \min\left(1, 2\frac{1}{N}\right)$. Letzteres zwingt den Algorithmus mindestens $2/N$ Satzbelege mit $w_i>0$ zu ermitteln.



## Weitere Erläuterungen
Wenn Ökonomen   von der [''Verwendung knapper Ressourcen''](https://www.dwds.de/r/?q=knappe+Ressourcen&corpus=public&date-start=1950&date-end=2018&genre=Belletristik&genre=Wissenschaft&genre=Gebrauchsliteratur&genre=Zeitung&format=full&sort=date_asc&limit=10) sprechen, dann ist damit die Nebenbedingung $\sum_{i=1}^N w_i = 1$ gemeint, auch ''budget constraint'' oder ''resource constraint'' genannt. Eine Maschine kann nur zu max. 100% ausgelastet werden [[Kan39, S.371](http://resistir.info/livros/kantorovich_mathematical_methods.pdf)]. Den Planeten Erde gibt es nur einmal als ''exhaustible resource'' [[Kop75, S.248/9](https://www.nobelprize.org/uploads/2018/06/koopmans-lecture.pdf)]. Und auch der [Aufmerksamkeitsökonomie](https://www.dwds.de/?q=Aufmerksamkeits%C3%B6konomie) des digitalen Zeitalters hat das Paar Augen, oder ein andere relevante Sinnesorgane einer Person, ultimativ nur begrenzte kognitive Kapazitäten, um Inhalte zu verarbeiten. Zum Beispiel, Satzbelege in einem Online-Wörterbuch. Analog können wir $w_i$ als Aufmerksamkeitsgewichtungen interpretieren. Die obere Schranke $u$ als die gewünschte maximale Aufmerksamkeit, die einem Satzbeleg zugestanden werden solle. 

## Softwareimplementierung
Wir verwenden den ''Sequential Least Squares Programming'' (SLSQP) Algorithmus aus dem [SciPy](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html) Paket [[Vir+20](https://doi.org/10.1038/s41592-019-0686-2)], der auf [[Kra88](http://degenerateconic.com/wp-content/uploads/2018/03/DFVLR_FB_88_28.pdf)] basiert.

In [1]:
import scipy.optimize
import numpy as np
from typing import List


def get_weights(goodness_scores: List[float], 
                similarity_matrix: List[List[float]],
                lam: float = 0.5,
                bnd_up: float = None
               ) -> (List[float], scipy.optimize.OptimizeResult):
    """Maximiere ''total goodness'' und minimiere ''total similarity''
    """
    # error checking
    if lam < 0:
        raise Exception(f"the preference lambda='{lam}' must be positive")
    if bnd_up <= 0 and bnd_up > 1:
        raise Exception(f"the upper bound must be 1.0>={bnd_up}>0.0")
    
    # specify the objective function of the optimization problem
    def objective_fn(w, *args):
        s = args[0]
        Q = args[1]
        return -np.dot(w, s) + np.sqrt(np.dot(w, np.dot(Q, w)))

    # how big is `N`
    n_examples = len(goodness_scores)
    
    # auto correct upper bound
    if (bnd_up is None) or (float(1.0 / n_examples) > bnd_up):
        bnd_up = min(2.0 * float(1.0 / n_examples), 1.0)
        print(f"upper bound changed to bnd_up={bnd_up}")

    # specify the non-negativity constraint >0, and set the upper bound
    con_lo_up = scipy.optimize.Bounds(0, bnd_up)
    
    # specify the "budget" constraint
    con_budget = {
        'type': 'eq', 
        'fun' : lambda x: np.sum(x) - 1
    }
    
    # set the intial values
    x0 = np.ones(n_examples) / n_examples
    
    # run the numerical optimization algorithm
    # - Trick: We can multipy `lam*Q` beforehand and save compute time!
    results = scipy.optimize.minimize(
        objective_fn, 
        x0=x0, 
        args=(goodness_scores, similarity_matrix * lam), 
        method='SLSQP', 
        bounds=con_lo_up, 
        constraints=[con_budget]
    )
    
    # done
    return results.x, results

# Goodness Scores and Similarity Scores
Die ''goodness scores'' und Ähnlichkeitsmetriken sind konstante Eingabeargumente für den Optimierungsalgorithmus und wurde mit den folgenden NLP Algorithmen vorab berechnet:

- Goodness scores: Linear classifier with BERT feature vectors
- Similarity scores: Siamese network distilled multilingual Sentence-BERT model [[RG20](https://doi.org/10.18653/v1/2020.emnlp-main.365)]. Code: https://github.com/UKPLab/sentence-transformers

## Demo Datensatz laden
Die gezippte JSON Datei liegt hier
https://drive.google.com/file/d/1ngh1wDw6OXVxzTakM9KBdkyNok_aebHZ/view?usp=share_link

In [2]:
# %%capture
!wget –load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget –quiet –save-cookies /tmp/cookies.txt –keep-session-cookies –no-check-certificate 'https://docs.google.com/uc?export=download&id=1ngh1wDw6OXVxzTakM9KBdkyNok_aebHZ' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/1n/p')&id=1ngh1wDw6OXVxzTakM9KBdkyNok_aebHZ" -O FILENAME && rm -rf /tmp/cookies.txt
!unzip /content/FILENAME
!rm FILENAME

--2023-02-17 14:59:59--  http://xn--quiet-xu3b/
Resolving xn--quiet-xu3b (xn--quiet-xu3b)... failed: Name or service not known.
wget: unable to resolve host address ‘xn--quiet-xu3b’
--2023-02-17 14:59:59--  http://xn--save-cookies-j19f/
Resolving xn--save-cookies-j19f (xn--save-cookies-j19f)... failed: Name or service not known.
wget: unable to resolve host address ‘xn--save-cookies-j19f’
/tmp/cookies.txt: Scheme missing.
--2023-02-17 14:59:59--  http://xn--keep-session-cookies-2t2l/
Resolving xn--keep-session-cookies-2t2l (xn--keep-session-cookies-2t2l)... failed: Name or service not known.
wget: unable to resolve host address ‘xn--keep-session-cookies-2t2l’
--2023-02-17 14:59:59--  http://xn--no-check-certificate-2t2l/
Resolving xn--no-check-certificate-2t2l (xn--no-check-certificate-2t2l)... failed: Name or service not known.
wget: unable to resolve host address ‘xn--no-check-certificate-2t2l’
--2023-02-17 14:59:59--  https://docs.google.com/uc?export=download&id=1ngh1wDw6OXVxzTakM9

Die JSON Datei funktioniert wie eine Key-Value Datenbank.

In [3]:
import json

with open("/content/tr-210-demo-dataset-1.json", "r") as fp:
    db = json.load(fp)

print(list(db.keys()))

['Piercing', 'Hosenträger', 'Rolli', 'Leggings', 'Look', 'Concealer', 'Sneaker', 'Jumpsuit', 'Blazer', 'Hoodie', 'Wimpernzange', 'Sweater']


## Hilfsfunktionen
Hilfsfunktion um die Satzbelege mit den Top-N ''goodness scores'' zu filtern. In der Matrix müssen die entprechenden Zeilen und Spalten gelöscht werden.

In [4]:
# filter top-N
def filter_top_n(good_vec, simi_mat, sent_ids, sent_text, N=10):
    indices = np.argsort(-good_vec)[:N]
    return good_vec[indices], simi_mat[indices, :][:, indices], np.array(sent_ids)[indices], np.array(sent_text)[indices]

# Playground
Versuche die folgenden Parameter zu ändern, 
und beobachte ob die Ergebnisse sich verbesser oder verschlechtern.

- `lemma`: Benutze ein anderes Stichwort
- `topn`: Berücksichte mehr/weniger Satzbelege mit Top-N ''goodness score''  (Vorsicht es ist $\mathcal{O}(N^2)$ komplex!)
- `lam`: Erhöhe/Senke die Präferenz für diversere Satzbelege 

In [5]:
from IPython.display import HTML, display
import tabulate
import numpy as np

lemmata = ['Piercing', 'Hosenträger', 'Rolli', 'Leggings', 'Look', 'Concealer', 'Sneaker', 'Jumpsuit', 'Blazer', 'Hoodie', 'Wimpernzange', 'Sweater']
print(lemmata)

['Piercing', 'Hosenträger', 'Rolli', 'Leggings', 'Look', 'Concealer', 'Sneaker', 'Jumpsuit', 'Blazer', 'Hoodie', 'Wimpernzange', 'Sweater']


In [11]:
# HIER ETWAS ÄNDERN
lemma = 'Piercing'  # Probiere ein anderes Stichwort
lam = 10.0  # kleine Wert `0<lam<1` wenn goodness scores wichtiger sind, `lam>1` um diversere Satzbelege zu favorisieren 

# ERSTMAL SO LASSEN
topn = 50  # begrenze die Anzahl von Satzbelge auf die Top-N goodness scores
bnd_up = 0.1  # Maximale Aufmerksamt `bnd_up*100 %`


# Daten auslesen, filtern
good_vec = np.array(db[lemma].get("goodness"))
simi_mat = np.array(db[lemma].get("similarity").get("sentence-semantics"))
sent_ids = np.array(db[lemma].get("sent_ids"))
sent_text = np.array(db[lemma].get("sent_text"))

# Daten filtern
good_vec, simi_mat, sent_ids, sent_text = filter_top_n(
    good_vec, simi_mat, sent_ids, sent_text, N=topn)

# Optimierung durchführen
%time weights, info = get_weights(good_vec, simi_mat, lam=lam, bnd_up=bnd_up)
print("\n")

"""
# Darstellung 1
indices = np.argsort(-weights)
for i in indices:
    if weights[i] > 1e-8:
        print(f"{good_vec[i]:6.3f} | {weights[i]:6.3f} | '{sent_text[i][:60]}'")
"""

# Darstellung 2
indices = np.argsort(-weights)
table = []
for i in indices:
    if weights[i] > 1e-8:  # Nur Satzbelege anzeigen mit w_i>0  (1e-8 hat numerische Gründe) 
        table.append([f"{good_vec[i]:6.3f}", f"{weights[i]:6.3f}", sent_text[i]])

display(HTML(tabulate.tabulate(table, tablefmt='html')))

CPU times: user 35.8 ms, sys: 0 ns, total: 35.8 ms
Wall time: 35.8 ms




0.971,0.1,"Fine Piercing Pop-Up, 17. und 18.01., Volume Gallery in Berlin-Mitte"
0.97,0.1,DAS It-Piercing aus dem Jahr 2018 bleibt auch 2019 noch Trend.
0.972,0.1,Wir haben uns durch die coolsten Instagram-Profile geklickt und die schönsten Piercing-Trends für euch herausgesucht ?
0.339,0.096,"Ich durfte nix unterschreiben, nicht einfach mal alleine eine Wohnung mieten oder ein Piercing machen lassen."
0.348,0.074,Die Kosten für ein Medusa-Piercing liegen in der Regel zwischen 40 und 50 Euro.
0.318,0.049,Für das Piercing müsst ihr natürlich nicht extra nach New York City zu Maria Tash fliegen.
0.29,0.047,Der Heilungsprozess dauert hier allerdings etwas länger: Rechnen Sie mit etwa zwei bis vier Monaten. Eine ebenfalls beliebte Variante ist das Klitorisvorhaut-Piercing.
0.344,0.038,"Ein besonderer Hingucker war das Nippel-Piercing der 27-Jährigen, das unter dem beigen Stoff durchschimmerte."
0.297,0.037,"Diese Stelle gilt seit der Antike als besonders erotische Zone, weshalb auch das Medusa-Piercing richtig sexy aussieht – erst recht in Kombination mit roten Lippen."
0.388,0.037,Beim Hair Piercing werden wahlweise Metall- oder Plastikringe in die Flechtfrisur integriert.
0.324,0.032,Für das „Heartilage“-Piercing wird die kleine Kugel der sonst typischen Helix-Ringe entfernt und ein kleines Herz geformt.
